<a href="https://colab.research.google.com/github/R4HUL-ROY/Multimodal_Results/blob/main/Multimodal_MobileNetv2_and_Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dependency

In [1]:
!pip install -q tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

     / 668.3 MB 9.1 MB/s
     |████████████████████████████████| 462 kB 8.8 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 43 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 8s (54.9 MB/s)
(Reading database ... 155632 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155610 files and directories currently installed.)
Preparing to unpack

In [2]:
!pip install -q tensorflow_text
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import pathlib
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, Adamax

     |████████████████████████████████| 4.6 MB 7.7 MB/s 
     |████████████████████████████████| 511.7 MB 5.0 kB/s 
     |████████████████████████████████| 438 kB 23.6 MB/s 
     |████████████████████████████████| 5.8 MB 32.0 MB/s 
     |████████████████████████████████| 1.6 MB 40.8 MB/s 


# Read the data

In [ ]:
# img_data_root = pathlib.Path('/content/drive/MyDrive/Tobacco3482-jpg/')
# print(img_data_root)
# for item in img_data_root.iterdir():
#   print(item)

# print()

# text_data_root = pathlib.Path('/content/drive/MyDrive/tobaco_OCR/')
# print(text_data_root)
# for item in text_data_root.iterdir():
#   print(item)

In [ ]:
# def get_corresponding_txtpath(img_path):
#     return img_path.replace("Tobacco3482-jpg", "tobaco_OCR")[:-3] + "txt"
    

# def get_file_paths_and_labels(img_data_root, text_data_root):
#      img_paths = [str(path) for path in img_data_root.glob('*/*.jpg')]
#      text_paths = [get_corresponding_txtpath(this_path) for this_path in img_paths]
#      img_labels = [p.split("/")[-2] for p in img_paths]
#      text_labels = [p.split("/")[-2] for p in text_paths]
#      return img_paths, img_labels, text_paths, text_labels

# img_paths, img_labels, text_paths, text_labels = get_file_paths_and_labels(img_data_root, text_data_root)
# print(len(img_paths))
# print(len(img_labels))
# print(len(text_paths))
# print(len(text_labels))

In [ ]:
# def get_text_from_path(path):
#     with open(path) as f:
#         lines = f.readlines()
#         lines  = ' '.join(lines)
#         f.close()
#     return lines

# out_text = get_text_from_path('/content/drive/MyDrive/tobaco_OCR/ADVE/0000435350.txt') 
# print(out_text)

In [ ]:
# text_contains = []
# c= 0 
# for i, this_path in enumerate(text_paths):
#     text_contains.append(get_text_from_path(this_path))
#     print(c, end= " ")
#     c +=1

In [ ]:
# df = pd.DataFrame(list(zip(text_paths, text_contains, img_paths, img_labels)),
#                columns =['text_paths','texts', 'img_paths', 'data_label'])

# # Merged Dataframe
# df.head()

In [ ]:
# df.to_csv("merged_data_for_multimodal_model.csv")

# read from saved csv file

In [4]:
df = pd.read_csv("/content/drive/MyDrive/project_resources/merged_data_for_multimodal_model.csv")
print(len(df))
df.head()

3482


,text_paths,texts,img_paths,data_label
0,/content/drive/MyDrive/tobaco_OCR/ADVE/0000136...,\n \n \n \n A Mpertant as yar\n sesiye teaeter...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,ADVE
1,/content/drive/MyDrive/tobaco_OCR/ADVE/0000435...,\n TE che fitm\n m66400 7127\n KOOLS are the o...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,ADVE
2,/content/drive/MyDrive/tobaco_OCR/ADVE/0030049...,SR Onrel ules cee\n Nee dss\n The one tales WT...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,ADVE
3,/content/drive/MyDrive/tobaco_OCR/ADVE/0000556...,so ARN Rr nr\n BWR Ga ||\n Vending Operators\n...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,ADVE
4,/content/drive/MyDrive/tobaco_OCR/ADVE/0349627...,\n &\n BR. :\n er non\n be 4\n op Re eo eee ee...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/03...,ADVE


In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['data_label']= le.fit_transform(df['data_label'])
df.head()

,text_paths,texts,img_paths,data_label
0,/content/drive/MyDrive/tobaco_OCR/ADVE/0000136...,\n \n \n \n A Mpertant as yar\n sesiye teaeter...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
1,/content/drive/MyDrive/tobaco_OCR/ADVE/0000435...,\n TE che fitm\n m66400 7127\n KOOLS are the o...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
2,/content/drive/MyDrive/tobaco_OCR/ADVE/0030049...,SR Onrel ules cee\n Nee dss\n The one tales WT...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
3,/content/drive/MyDrive/tobaco_OCR/ADVE/0000556...,so ARN Rr nr\n BWR Ga ||\n Vending Operators\n...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
4,/content/drive/MyDrive/tobaco_OCR/ADVE/0349627...,\n &\n BR. :\n er non\n be 4\n op Re eo eee ee...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/03...,0


In [6]:
label_dict = {}
for idx, row in enumerate(df['text_paths']):
    path = df.at[idx, 'text_paths']
    lab = path.split("/")[-2]
    label_dict[str(lab)] = df.at[idx, 'data_label']
    
print(label_dict) 

{'ADVE': 0, 'Form': 2, 'Note': 6, 'Email': 1, 'News': 5, 'Resume': 8, 'Scientific': 9, 'Memo': 4, 'Report': 7, 'Letter': 3}


In [7]:
import re
def preprocess(text_string):
    preprocessed_string = re.sub(r'[^\w\s]','',text_string)
    preprocessed_string = preprocessed_string.replace('\n',' ')
    preprocessed_string = preprocessed_string.replace('_',' ')
    preprocessed_string = re.sub(' +', ' ', preprocessed_string)
    return preprocessed_string

In [8]:
# Tokenize, Lemmatize, stopwords removal
import spacy 
import nltk
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = stopwords.words("english")

def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

normalize("counting playing the Home", lowercase=True, remove_stopwords=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'count play home'

In [9]:
df['texts'] = [preprocess(str(this_text)) for this_text in df['texts']]
df['texts'] = [normalize(this_text, lowercase=True, remove_stopwords=True) for this_text in df['texts']]
df.head()

,text_paths,texts,img_paths,data_label
0,/content/drive/MyDrive/tobaco_OCR/ADVE/0000136...,mpertant yar sesiye teaetered cabiieess baely ...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
1,/content/drive/MyDrive/tobaco_OCR/ADVE/0000435...,te che fitm m66400 7127 kools cigarette taste ...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
2,/content/drive/MyDrive/tobaco_OCR/ADVE/0030049...,sr onrel ules cee nee dss one tale wt lower ta...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
3,/content/drive/MyDrive/tobaco_OCR/ADVE/0000556...,arn rr nr bwr ga vend operator column worth 8 ...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/00...,0
4,/content/drive/MyDrive/tobaco_OCR/ADVE/0349627...,br er non 4 op eo eee ee eee talk smoke decide...,/content/drive/MyDrive/Tobacco3482-jpg/ADVE/03...,0


# Download and load glove model

In [ ]:
# import urllib.request
# urllib.request.urlretrieve('https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip')
# !unzip "/content/glove.6B.zip" -d "/content/"

In [ ]:
# emmbed_dict = {}
# with open('/content/glove.6B.200d.txt','r') as f:
#   for line in f:
#     values = line.split()
#     word = values[0]
#     vector = np.asarray(values[1:],'float32')
#     emmbed_dict[word]=vector
# f.close()

# Load the predownloaded glove vector from drive

In [10]:
emmbed_dict = {}
with open('/content/drive/MyDrive/project_resources/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector
f.close()

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Multimodal utility function

In [12]:
def dataframe_to_dataset(dataframe):
    d = {}
    embed_tensor = []
    for i in dataframe['texts_embedding']:
        embed_tensor.append(tf.convert_to_tensor(i))

    img_path_tensor= []
    for i in dataframe['img_paths']:
        img_path_tensor.append(tf.convert_to_tensor(i))

    d['texts_embedding'] = embed_tensor
    d['img_paths'] = img_path_tensor

    labels = dataframe["data_label"]
    labels = tf.convert_to_tensor(labels)
    ds = tf.data.Dataset.from_tensor_slices((d, labels))
    return ds

In [13]:
def preprocess_image(image_path):
    extension = tf.strings.split(image_path)[-1]

    image = tf.io.read_file(image_path)
    if extension == b"jpg":
        image = tf.image.decode_jpeg(image, 3)
    else:
        image = tf.image.decode_png(image, 3)
    image = tf.image.resize(image, (224, 224))
    # per image standardization
    return image


def preprocess_text(sample):
    pass

def preprocess_text_and_image(sample):
    image = preprocess_image(sample["img_paths"])
    text = sample['texts_embedding']
    return {"image_inputs": image,  "text_inputs": text}

In [14]:
batch_size = 32
auto = tf.data.AUTOTUNE

def prepare_dataset(dataframe, training = True):
    ds = dataframe_to_dataset(dataframe)
    if training:
        ds = ds.shuffle(len(train_df))
    ds = ds.map(lambda x, y: (preprocess_text_and_image(x), y)).cache()
    ds = ds.batch(batch_size).prefetch(auto)
    return ds

# Text encoder and vision encoder Utility Function for model Creation

In [15]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

In [16]:
def create_text_encoder(text_projection_dims, dropout_rate, text_model_trainable):

    inputs = keras.Input(shape=(500,), dtype=tf.int32, name="text_inputs")

    embed = tf.keras.layers.Embedding(  num_words,
                                        200,
                                        embeddings_initializer = Constant(embedding_matrix),
                                        trainable = text_model_trainable
                                     )(inputs)
    

    x = tf.keras.layers.Dropout(0.2)(embed)
    x = tf.keras.layers.Conv1D(filters=128,kernel_size=3,padding='valid',activation='relu',strides=1)(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Conv1D(filters=128,kernel_size=3,padding='valid',activation='relu',strides=1)(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Conv1D(filters=128,kernel_size=3,padding='valid',activation='relu',strides=1)(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Conv1D(filters=128,kernel_size=3,padding='valid',activation='relu',strides=1)(x)
    x = tf.keras.layers.GlobalMaxPooling1D()(x)
    outputs = keras.layers.Dense(units = text_projection_dims, activation = "relu")(x)

    return keras.Model(inputs, outputs, name="text_encoder")

In [17]:
def create_vision_encoder(img_projection_dims, dropout_rate, img_base_model_trainable):
    img_shape=(224,224,3)
    seed_value = 42

    base_model=tf.keras.applications.MobileNetV2(include_top=False,
                                                 input_shape=img_shape, 
                                                 pooling='avg', 
                                                 classes=10,
                                                 weights=None)
    
    base_model.load_weights("/content/drive/MyDrive/project_resources/Imagenet_weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5")
    base_model.trainable = img_base_model_trainable


    inputs = keras.Input(shape=img_shape)
    x = base_model(inputs, training = img_base_model_trainable)

    outputs =tf.keras.layers.Dense(img_projection_dims, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006), 
                         bias_regularizer=regularizers.l1(0.006),activation='relu',kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value))(x)

    model=Model(inputs, outputs)

    # Receive the images as inputs.
    image_inputs = keras.Input(shape=(224, 224, 3), name="image_inputs")
    preprocessed_image = tf.keras.applications.mobilenet_v2.preprocess_input(image_inputs)
    
    embeddings = model(preprocessed_image)

    # Create the vision encoder model.
    return keras.Model(image_inputs, embeddings, name="vision_encoder")

In [18]:
def create_multimodal_model(**kwargs):

    img_projection_dims = kwargs['img_projection_dims']
    text_projection_dims = kwargs['text_projection_dims']
    dropout_rate = kwargs['dropout_rate']
    img_base_model_trainable = kwargs['img_base_model_trainable']
    text_model_trainable = kwargs['text_model_trainable']

    # Receive the images and text as inputs.
    image_inputs = keras.Input(shape=(224, 224, 3), name="image_inputs")
    
    text_inputs = keras.Input(shape=(500,), dtype=tf.int32, name="text_inputs")

    # Create the encoders.
    vision_encoder = create_vision_encoder(img_projection_dims, dropout_rate, img_base_model_trainable)
    text_encoder = create_text_encoder(text_projection_dims, dropout_rate, text_model_trainable)


    # Fetch the embedding projections.
    vision_projections = vision_encoder(image_inputs)
    text_projections = text_encoder(text_inputs)

    # Concatenate the projections and pass through the classification layer.
    concatenated = keras.layers.Concatenate()([vision_projections, text_projections])

    # dropped_concat = tf.keras.layers.Dropout(dropout_rate)(concatenated)

    outputs = keras.layers.Dense(10, activation="softmax")(concatenated)
    return keras.Model([image_inputs, text_inputs], outputs)

# Trial train and evaluating result

In [19]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import random
random.seed(56)

In [22]:
epochs = 2
fine_tune_epochs = 2
lr = 0.001

d = {
    "precision" : [],
     "recall" : [],
     "f1_score" : [],
     "accuracy" : [],
     "seed_value" : []
}
seed = []

for i in range(2):
    print('#Run: ', i+1)
    sd = random.randint(0, 100)
    seed.append(sd)

    # Split the dataframe
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=sd)
    test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=sd)

    max_len = 500
    # Fit the tokenizer
    tokenizer = Tokenizer(num_words=100000)
    tokenizer.fit_on_texts(train_df['texts'])

    # sequence the input corpus and add zero padding upto 500 word
    train_sequence = tokenizer.texts_to_sequences(train_df['texts'])
    train_padded = pad_sequences(train_sequence, maxlen = max_len, truncating = "post", padding = "post" )

    valid_sequence = tokenizer.texts_to_sequences(val_df['texts'])
    valid_padded = pad_sequences(valid_sequence, maxlen = max_len, truncating = "post", padding = "post" )

    test_sequence = tokenizer.texts_to_sequences(test_df['texts'])
    test_padded = pad_sequences(test_sequence, maxlen = max_len, truncating = "post", padding = "post" )

    train_tensor = [tf.convert_to_tensor(train_padded[i]) for i in range(train_padded.shape[0])]
    train_df['texts_embedding'] = train_tensor

    test_tensor = [tf.convert_to_tensor(test_padded[i]) for i in range(test_padded.shape[0])]
    test_df['texts_embedding'] = test_tensor   

    val_tensor = [tf.convert_to_tensor(valid_padded[i]) for i in range(valid_padded.shape[0])]
    val_df['texts_embedding'] = val_tensor 


    # Preparing dataset
    train_ds = prepare_dataset(train_df)
    val_ds = prepare_dataset(val_df, False)
    test_ds = prepare_dataset(test_df, False)

    # Creating Glove embedding matrix
    word_index = tokenizer.word_index
    num_words = len(word_index) + 1
    embedding_matrix = np.zeros((num_words, 200))

    for word, i in word_index.items():
        if i < num_words:
            emb_vec = emmbed_dict.get(word)
            if emb_vec is not None:
                embedding_matrix[i] = emb_vec


    # Creating the multimodal model
    multimodal_model = create_multimodal_model( img_projection_dims=512,
                                            text_projection_dims=128,
                                            dropout_rate=0.2,
                                            img_base_model_trainable=False,
                                            text_model_trainable= False
                                           )
    keras.utils.plot_model(multimodal_model, show_shapes=True)

    
    multimodal_model.compile(Adamax(learning_rate=lr), loss="sparse_categorical_crossentropy", metrics="accuracy")
    history = multimodal_model.fit(train_ds, validation_data=val_ds, epochs=epochs,verbose=2, initial_epoch = 0)

    # Fine Tune,  making the image base_model(MobileNetV2 here) Trainable = True 
    multimodal_model.layers[2].layers[3].layers[1].trainable = True

    
    total_epochs= epochs + fine_tune_epochs
    multimodal_model.compile(Adamax(learning_rate=lr/10), loss="sparse_categorical_crossentropy", metrics="accuracy")
    history_fine = multimodal_model.fit(train_ds, validation_data=val_ds, epochs=total_epochs, verbose=2,initial_epoch=epochs)

    y_pred = multimodal_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))

    d["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d['accuracy'].append(float(accuracy_score(y_true, y_pred)))


#Run:  1
Epoch 1/2
88/88 - 646s - loss: 6.9211 - accuracy: 0.5368 - val_loss: 3.8317 - val_accuracy: 0.6246 - 646s/epoch - 7s/step
Epoch 2/2
88/88 - 103s - loss: 2.9631 - accuracy: 0.6952 - val_loss: 2.4327 - val_accuracy: 0.6905 - 103s/epoch - 1s/step
Epoch 3/4
88/88 - 412s - loss: 2.3808 - accuracy: 0.6725 - val_loss: 2.1607 - val_accuracy: 0.7249 - 412s/epoch - 5s/step
Epoch 4/4
88/88 - 402s - loss: 1.9452 - accuracy: 0.7770 - val_loss: 1.9501 - val_accuracy: 0.7622 - 402s/epoch - 5s/step
11/11 [==============================] - 72s 6s/step
#Run:  2
Epoch 1/2
88/88 - 158s - loss: 7.0208 - accuracy: 0.5379 - val_loss: 3.9124 - val_accuracy: 0.6533 - 158s/epoch - 2s/step
Epoch 2/2
88/88 - 116s - loss: 3.0676 - accuracy: 0.6948 - val_loss: 2.4025 - val_accuracy: 0.7564 - 116s/epoch - 1s/step
Epoch 3/4
88/88 - 410s - loss: 2.3670 - accuracy: 0.7023 - val_loss: 2.0819 - val_accuracy: 0.7736 - 410s/epoch - 5s/step
Epoch 4/4
88/88 - 429s - loss: 1.9878 - accuracy: 0.7928 - val_loss: 1.8855

In [23]:
from google.colab import files
print(seed)
d['seed_value'] = seed
result = pd.DataFrame(d, index = ["run-"+str(i+1) for i in range(2)])
print(result)
result.to_csv("multimodal_result_mobileNetV2_glove.csv")

files.download('multimodal_result_mobileNetV2_glove.csv')

[60, 83]
       precision    recall  f1_score  accuracy  seed_value
run-1   0.810921  0.790230  0.785587  0.790230          60
run-2   0.834334  0.824713  0.823515  0.824713          83


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>